In [1]:
"""Library"""
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

missing_index = -99.99
col_name = "adj_close_excess"

/home/ling/miniconda3/envs/ap_tf2/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/ling/miniconda3/envs/ap_tf2/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
"""Load price data"""
price = pd.read_csv("../data/yprice_1mo.csv")
price["date"] = pd.to_datetime(price["date"])
price = price.set_index(["symbol", "date"])
price.head()

adj_close_excess
symbol date                        
3IN    2000-01-31            -99.99
       2000-02-29            -99.99
       2000-03-31            -99.99
       2000-04-30            -99.99
       2000-05-31            -99.99

In [3]:
"""Load factors"""
factors = pd.read_csv("../data/factors.csv")
factors["date"] = pd.to_datetime(factors["date"])
factors = factors.set_index("date")
factors.head()

,smb,hml,umd,rf,rm,rmrf
date,,,,,,
1980-10-31,0.000036,-0.072617,0.041297,0.011246,0.060245,0.048999
1980-11-30,0.006165,-0.018527,0.006964,0.010202,0.008133,-0.002069
1980-12-31,0.002356,-0.001671,0.015869,0.010252,-0.044098,-0.054350
1981-01-31,0.020263,0.005622,0.014245,0.009949,-0.005982,-0.015931
1981-02-28,-0.008055,0.022217,0.006087,0.009175,0.057703,0.048528


In [4]:
"""Join the two data"""
data = price.combine_first(factors)\
    .replace(-99.99, None)\
    .dropna(axis=0)\
    .astype("float")

data.head()

adj_close_excess       hml        rf        rm      rmrf  \
symbol date                                                                   
3IN    2007-03-31         -0.007107 -0.001285  0.004376  0.033447  0.029071   
       2007-04-30          0.029337 -0.001255  0.004452  0.024495  0.020044   
       2007-05-31         -0.046842  0.014259  0.004527  0.028190  0.023663   
       2007-06-30         -0.043623  0.005187  0.004686 -0.007571 -0.012257   
       2007-07-31         -0.048452 -0.020350  0.004669 -0.033235 -0.037904   

                        smb       umd  
symbol date                            
3IN    2007-03-31  0.001003  0.011036  
       2007-04-30 -0.001923  0.002628  
       2007-05-31 -0.017083  0.002056  
       2007-06-30 -0.052502 -0.047472  
       2007-07-31  0.026305  0.019840

In [5]:
"""One asset"""
mod = sm.OLS(data.loc["3IN", col_name],
             data.loc["3IN"].drop(columns=[col_name]).assign(const=1),
             missing=missing_index,
             hasconst=True
             )

res = mod.fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       adj_close_excess   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.051
Date:                Mon, 21 Mar 2022   Prob (F-statistic):              0.396
Time:                        13:51:11   Log-Likelihood:                 79.816
No. Observations:                 130   AIC:                            -145.6
Df Residuals:                     123   BIC:                            -125.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
hml           -0.2558      0.589     -0.434      0.665      -1.422       0.911
rf          2.411e+04    4.3e+06      0.006      0.996   -8.48e+06    8.53e+06
rm         -2.411e+04    4.3e+06     -0.006      0.996   -8.53e+06    8.48e+06
rmrf        2.411e+04    4.3e+06      0.006      0.996   -8.48e+06    8.53e+06
smb           -0.2179      0.382     -0.570      0.570      -0.974       0.538
umd           -0.6829      0.290     -2.353      0.020      -1.257      -0.108
const          0.0310      0.015      2.048      0.043       0.001       0.061
==============================================================================
Omnibus:                       50.227   Durbin-Watson:                   1.442
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1069.877
Skew:                           0.621   Prob(JB):                    4.78e-233
Kurtosis:                      16.999   Cond. No.                     6.31e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.27e-16. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [6]:
"""Sklearn Linear Regression"""
coef = {}

for symbol in data.index.get_level_values("symbol").unique():
    train = data.loc[symbol]
    lr = LinearRegression()\
        .fit(
        train.drop(columns=[col_name]),
        train.loc[:, col_name]
    )

    coef[symbol] = dict(zip(lr.feature_names_in_, lr.coef_))
    coef[symbol]["intercept"] = lr.intercept_

In [7]:
coef = pd.DataFrame(coef).T

coef

,hml,rf,rm,rmrf,smb,umd,intercept
3IN,-0.255756,2.410618e+04,-2.410742e+04,2.410753e+04,-0.217906,-0.682861,0.030956
888,-0.784311,2.848431e+06,-2.848441e+06,2.848442e+06,-0.364076,-0.584096,0.037441
88E,0.206763,-4.116761e+07,4.116784e+07,-4.116784e+07,-0.417815,0.095312,-0.093344
AAAP,0.491693,-1.731255e+07,1.731271e+07,-1.731270e+07,2.023388,-0.025854,-0.074771
AAIF,-0.086236,6.221672e+05,-6.221737e+05,6.221735e+05,-0.319593,-0.655647,0.036221
...,...,...,...,...,...,...,...
ZIOC,1.141257,-4.902695e+06,4.902445e+06,-4.902445e+06,-0.053421,-0.157455,0.053415
ZOL,2.243937,2.030292e+07,-2.030279e+07,2.030279e+07,0.309445,-0.555276,-0.043658
ZOO,-0.376585,5.792402e+05,-5.792688e+05,5.792688e+05,0.252810,-0.582461,0.054019
ZTF,0.092181,3.541116e+06,-3.541124e+06,3.541124e+06,0.162200,-0.314676,0.038444


In [8]:
coef["intercept"].abs().describe()

count    1020.000000
mean        0.066768
std         0.131189
min         0.000053
25%         0.025719
50%         0.036585
75%         0.050122
max         1.615055
Name: intercept, dtype: float64

In [9]:
# export results
coef.to_csv("../data/fama_results.csv")